In [ ]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [2]:
import src.rag as rag
import pickle
import src.comparator_sets as comparators
import glob 
import os
import time

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [3]:
with open('output/comparator-sets/schools.pkl', 'rb') as schools_file:
     all_schools = pickle.load(schools_file)

with open('output/comparator-sets/pupil_comparators.pkl', 'rb') as pupil_file:
     pupil_comparators = pickle.load(pupil_file)
     
with open('output/comparator-sets/building_comparators.pkl', 'rb') as building_file:
     building_comparators = pickle.load(building_file)

Select the school for the RAG calculation

In [10]:
start_time = time.time()

target_school = 103341
comparator_set = comparators.get_comparator_set_by(lambda s: s.index == target_school, all_schools, pupil_comparators).set_index('URN')

Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [11]:
rag.compute_rag(target_school, comparator_set)

## Processing Time

In [12]:
print(f'Processing Time: {time.time() - start_time} seconds')

Processing Time: 0.4898262023925781 seconds
